In [ ]:
import h5py
import numpy as np
import torch
import pywt
import csv
import torch.nn.functional as F
from torch import nn
from torch import optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
f_train = h5py.File('X_train_new.h5', 'r')
f_test = h5py.File('X_test_new.h5', 'r')
X_train = f_train['features']
X_test = f_test['features']

X_train = np.asarray(X_train)
X_test = np.asarray(X_test)

In [ ]:
y_train = [0] * 946

with open('y_train_AvCsavx.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
       # print(row['id'], row['label'])
       y_train[int(row['id'])] = int(row['label'])
    
    
#extend y_train labels to every segment (instead of every patient)
y_train_segm = np.repeat(y_train, 40)
y_train_segm = torch.from_numpy(y_train_segm)
X_train_sequences = np.reshape(X_train, (X_train.shape[0] * X_train.shape[1] * X_train.shape[2], X_train.shape[3]))


In [ ]:
# Feature extraction using wavelets

preprocessed = []

for i in range(X_train_sequences.shape[0]):
    x = X_train_sequences[i]
    wp = pywt.WaveletPacket(data=x, wavelet='db8', mode='symmetric')
    preprocessed.append(wp['aaad'].data)


preprocessed = np.asarray(preprocessed)
X_train_segm = np.reshape(preprocessed, (X_train.shape[0] * X_train.shape[1], X_train.shape[2], 
                                         preprocessed.shape[1]))
                    
X_train_segm = np.transpose(X_train_segm, (0, 2, 1))
X_train_segm = torch.from_numpy(X_train_segm)
print('X_train_segm shape', X_train_segm.shape)

In [ ]:
#X_train_segm = np.reshape(X_train, [X_train.shape[0] * X_train.shape[1], 1, X_train.shape[3], X_train.shape[2]])

# train/val split
nr_train_examples = X_train_segm.shape[0]
print('nr train examples', nr_train_examples)
X_train_splitted = X_train_segm[:-( nr_train_examples // 10)]
X_val_splitted = X_train_segm[-(nr_train_examples // 10):]


# data normalization
mean = torch.mean(X_train_splitted, (0, 1), keepdim=True)
std = torch.std(X_train_splitted, (0, 1), keepdim=True)
X_train_splitted = (X_train_splitted - mean) / (std + 1e-8)
X_train_splitted = X_train_splitted.to(device=device)
X_val_splitted = (X_val_splitted - mean) / (std + 1e-8)
X_val_splitted = X_val_splitted.to(device=device)
print('X_train_splitted.shape', X_train_splitted.shape)
print('X_val_splitted.shape', X_val_splitted.shape)


y_train_splitted = y_train_segm[:-(nr_train_examples // 10)].to(device=device)
y_val_splitted = y_train_segm[-(nr_train_examples // 10):].to(device=device)
print('y_train_splitted.shape', y_train_splitted.shape)
print('y_val_splitted.shape', y_val_splitted.shape)


X_train_concat = X_train_splitted #torch.from_numpy(X_train_splitted).to(device=device)
y_train_concat = y_train_splitted #torch.from_numpy(y_train_splitted).to(device=device)

nr_cls_1_train = torch.sum(y_train_concat).item()
nr_cls_0_train =  y_train_concat.shape[0] - nr_cls_1_train

print("Nr_cls_1_train = ", nr_cls_1_train)
print("Nr_cls_0_train = ", nr_cls_0_train)


In [ ]:
class LSTM_EEG(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers, drop_prob=0.):
        super(LSTM_EEG, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, dropout=drop_prob, 
                            batch_first=True, bidirectional=False)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, 2, bias=True)
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        lstm_out, hidden = self.lstm(x, hidden)
        lstm_out = lstm_out[:, -1, :]
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        return out
    
    def init_hidden(self, batch_size):
        hidden_state = torch.randn(self.n_layers, batch_size, self.hidden_dim).to(device)
        cell_state = torch.randn(self.n_layers, batch_size, self.hidden_dim).to(device)
        hidden = (hidden_state, cell_state)
        return hidden

In [ ]:
#https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

net = LSTM_EEG(input_size = X_train_segm.shape[2], hidden_dim=128, n_layers=2)

net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = 0.005)

batch_size = 64 #32
print_every_n = 200

for epoch in range(100):  
    perm = np.random.RandomState(seed=42).permutation(X_train_concat.shape[0])    
    perm = torch.from_numpy(perm).to(device=device)
    
    X_train_concat = X_train_concat[perm]#.to(device)
    y_train_concat = y_train_concat[perm]#.to(device)
    
    running_loss = 0.0
    total_acc = 0.0
    acc_print = 0.0
    
    for batch_i in range(X_train_concat.shape[0] // batch_size):
        inputs = X_train_concat[batch_i * batch_size:(batch_i+1) * batch_size]
        labels = y_train_concat[batch_i * batch_size:(batch_i+1) * batch_size]

        optimizer.zero_grad()
        hidden = net.init_hidden(batch_size)
        outputs = net(inputs, hidden)
                
        loss = criterion(outputs, labels)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(net.parameters(), 1.)

        optimizer.step()
        
        pred = outputs.data.max(1, keepdim=True)[1]
        correct = pred.eq(labels.view_as(pred)).cuda().sum()
        acc = ((correct * (1.0)) / ((1.0) * labels.shape[0]))
        total_acc += acc.item()
        acc_print += acc.item()

        # print statistics
        running_loss += loss.item()
        if batch_i % print_every_n == 0:   
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, batch_i + 1, running_loss / print_every_n))

            print('[%d, %5d] acc: %.3f' %
                  (epoch + 1, batch_i + 1, acc_print / print_every_n))           
            
            running_loss = 0.0
            acc_print = 0.0
            
        optimizer.zero_grad()
    
    print("Accuracy train epoch ", epoch, ": ", total_acc /  (X_train_concat.shape[0] // batch_size))

    total_acc_val = 0.0
    total_loss_val = 0.0
    batch_size_val = 16
    
    with torch.no_grad():
        for batch_i in range(X_val_splitted.shape[0] // batch_size_val):
            
            optimizer.zero_grad()

            inputs_val = X_val_splitted[batch_i * batch_size_val:(batch_i+1) * batch_size_val]#.to(device)
            labels_val = y_val_splitted[batch_i * batch_size_val:(batch_i+1) * batch_size_val]#.to(device)
            
            hidden = net.init_hidden(batch_size_val)
            outputs_val = net(inputs_val, hidden)            
            
            '''
            # Thresholding
            
            outputs_val = F.softmax(outputs_val, dim = 1)
            
            outputs_val[:, 0] = outputs_val[:, 0] * ((nr_cls_0_train * 1.0) + nr_cls_1_train) / (1.0 * nr_cls_0_train)
 
            outputs_val[:, 1] = outputs_val[:, 1] * ((nr_cls_0_train * 1.0) + nr_cls_1_train) / (1.0 * nr_cls_1_train)
    
                    
            temp = outputs_val[:, 0]
            
            outputs_val[:, 0] = temp / (temp + outputs_val[:, 1])
            
            outputs_val[:, 1] = 1 - outputs_val[:, 0] 
            
            
            print('outputs_val', outputs_val[0:10])
            '''
            
            pred = outputs_val.data.max(1, keepdim=True)[1]
            
            print('pred: ', pred[:10])
            
            correct = pred.eq(labels_val.view_as(pred)).cuda().sum()
            acc_val = ((correct * (1.0)) / ((1.0) * labels_val.shape[0]))
            loss_val = criterion(outputs_val, labels_val).item()
            total_acc_val += acc_val
            total_loss_val += loss_val

        print("Accuracy val epoch: ", epoch, ": ", (total_acc_val /  (X_val_splitted.shape[0] // batch_size_val)))
        print("Loss val epoch ", epoch, ": ", total_loss_val / (X_val_splitted.shape[0] // batch_size_val))


print('Finished Training')

In [ ]:
X_test_sequences = np.reshape(X_test, (X_test.shape[0] * X_test.shape[1] * X_test.shape[2], X_test.shape[3]))

preprocessed = []

for i in range(X_test_sequences.shape[0]):
    x = X_test_sequences[i]
    wp = pywt.WaveletPacket(data=x, wavelet='db8', mode='symmetric')
    preprocessed.append(wp['aaad'].data)

    
preprocessed = np.asarray(preprocessed)
X_test_segm = np.reshape(preprocessed, (X_test.shape[0] * X_test.shape[1], X_test.shape[2], 
                                         preprocessed.shape[1]))
                    
X_test_segm = np.transpose(X_test_segm, (0, 2, 1))

In [ ]:
X_test_segm = torch.from_numpy(X_test_segm)
X_test_segm = (X_test_segm - mean) / (std + 1e-8)
X_test_segm = X_test_segm.to(device)

batch_size = 40
preds = []

with torch.no_grad():
    for batch_i in range(X_test_segm.shape[0] // batch_size):
        inputs = X_test_segm[batch_i * batch_size:(batch_i+1) * batch_size]

        hidden = net.init_hidden(batch_size)

        outputs_test = F.softmax(net(inputs, hidden), dim=1) #added item()
        
        '''
        outputs_test[:, 0] = outputs_test[:, 0] * ((nr_cls_0_train * 1.0) + nr_cls_1_train) / (1.0 * nr_cls_0_train)

        outputs_test[:, 1] = outputs_test[:, 1] * ((nr_cls_0_train * 1.0) + nr_cls_1_train) / (1.0 * nr_cls_1_train)

        temp = outputs_test[:, 0]

        outputs_test[:, 0] = temp / (temp + outputs_test[:, 1])

        outputs_test[:, 1] = 1 - outputs_test[:, 0]
        '''
        
        preds += outputs_test.data[:, 1]
    

preds = np.asarray(preds)
preds = np.reshape(preds, (-1, 40))
preds = np.mean(preds, axis=1)
preds = (np.sign(preds-0.5) + 1)//2
preds = [int(i) for i in preds]

out = open('LSTM_baseline.csv', "w")
out.write("id,label\n")
rows = ['']*len(preds)
for num in range(len(preds)):
    rows[num]='%d,%d\n' % (num, preds[num])
out.writelines(rows)
out.close()